In [2]:
import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

## Module

In [3]:
def generate_coefficientCodes_at(current, variablePos, maxPower):
  if variablePos < len(current):
    current[variablePos] = 0
    while sum(current) <= maxPower:
      for v in generate_coefficientCodes_at(current, variablePos + 1, maxPower):
        yield v
      current[variablePos] = current[variablePos] + 1
    current[variablePos] = 0
  else:
    yield current.copy()
  
def generate_coefficientCodes(variablesCount, maxPower):
  current = np.full(variablesCount, 0)
  return generate_coefficientCodes_at(current, 0, maxPower)

def coefficientCode_to_String(code):
  return np.array2string(np.array(code), separator='_')[1:-1]

def generate_model(variablesCount, maxPower, random_low=1, random_high=10, random_state=None):
  coefficients_names = [coefficientCode_to_String(v) for v in generate_coefficientCodes(variablesCount, maxPower)]
  coefficients_values = np.random.default_rng(random_state).integers(low=random_low, high=random_high, size=len(coefficients_names))
  return pd.DataFrame([coefficients_values], columns=coefficients_names)

model = generate_model(variablesCount=2, maxPower=3, random_state=13)
model

,0_0,0_1,0_2,0_3,1_0,1_1,1_2,2_0,2_1,3_0
0,9,8,8,8,1,8,9,3,2,1


In [4]:
9*1*1 + 8*1*3 + 8*1*9 + 8*1*27 + 1*2*1 + 8*2*3 + 9*2*9 + 3*4*1 + 2*4*3 + 1*8*1

577

In [5]:
def coefficientString_to_code(value):
  return [int(v) for v in value.split('_')]

def calculate_polynomial_term(coefficient, values, valuePowers):
  result = coefficient
  for i in range(len(values)):
    result = result * pow(values[i], valuePowers[i])
  return result

class ExecutionModel:

  def __init__(self, model):
    self.model = model
    self.coefficients = model.iloc[0].to_numpy()
    self.valuePowers = [coefficientString_to_code(c) for c in model.columns]

  def calculate_result(self, values):
    return sum(
      calculate_polynomial_term(self.coefficients[i], values, self.valuePowers[i]) 
      for i in range(len(self.coefficients)))
  
  def get_inputs_count(self):
    return len(self.valuePowers[0])
  
  def generate_data_set(self, points_count, scale=10, random_state=None):    
    inputCount = self.get_inputs_count()
    inputs = np.random.default_rng(random_state).random((points_count, inputCount)) * scale
    results = [self.calculate_result(inputs[i]) for i in range(points_count)]
    data = np.concatenate((inputs, np.array([results]).T), axis=1)
    columns = [i for i in chain((chr(ord('a')+i) for i in range(inputCount)), ['result'])]
    return pd.DataFrame(data, columns=columns)

exModel = ExecutionModel(model)
exModel.calculate_result([2, 3])

577

In [6]:
exModel.generate_data_set(2, random_state=13)

,a,b,result
0,8.647976,8.553025,14112.723520
1,8.110234,2.614464,1978.962909


## Export

In [10]:
random_state = 42
points_count = 10000

for variablesCount in range(1, 7):
  for maxPower in range(1, 9):
    model = generate_model(variablesCount=variablesCount, maxPower=maxPower, random_state=random_state)
    data = ExecutionModel(model).generate_data_set(points_count, random_state=random_state)
    
    fileName = f'003_poly_{variablesCount}_{maxPower}.h5'
    model.to_hdf(fileName, "model", format="table")
    data.to_hdf(fileName, "data", format="table")
    print(fileName)

003_poly_1_1.h5
003_poly_1_2.h5
003_poly_1_3.h5
003_poly_1_4.h5
003_poly_1_5.h5
003_poly_1_6.h5
003_poly_1_7.h5
003_poly_1_8.h5
003_poly_2_1.h5
003_poly_2_2.h5
003_poly_2_3.h5
003_poly_2_4.h5
003_poly_2_5.h5
003_poly_2_6.h5
003_poly_2_7.h5
003_poly_2_8.h5
003_poly_3_1.h5
003_poly_3_2.h5
003_poly_3_3.h5
003_poly_3_4.h5
003_poly_3_5.h5
003_poly_3_6.h5
003_poly_3_7.h5
003_poly_3_8.h5
003_poly_4_1.h5
003_poly_4_2.h5
003_poly_4_3.h5
003_poly_4_4.h5
003_poly_4_5.h5
003_poly_4_6.h5
003_poly_4_7.h5
003_poly_4_8.h5
003_poly_5_1.h5
003_poly_5_2.h5
003_poly_5_3.h5
003_poly_5_4.h5
003_poly_5_5.h5
003_poly_5_6.h5
003_poly_5_7.h5
003_poly_5_8.h5
003_poly_6_1.h5
003_poly_6_2.h5
003_poly_6_3.h5
003_poly_6_4.h5
003_poly_6_5.h5
003_poly_6_6.h5
003_poly_6_7.h5
003_poly_6_8.h5
